# PROCESO ETL (Extract, Transform, Load)

In [15]:
# importar bibliotecas (intentar lo minimo indispensable)

import pandas as pd
import numpy as np
import json

<br>

## PARTE 1 - DATASET STEAM_GAMES
### OBJETIVOS
* Cargar el dataset output_steam_games.json
* Realizar análisis básicos y transformaciones según se requiera para el MVP
* Exportar dataframes en formato parquet

<br><br>

### 1. Cargar el dataset 'output_steam_games.json'

<br><br>


In [2]:
archivo = open("datasets/output_steam_games.json", "r")
lista = []

for linea in archivo:
    linea = linea.strip() # Eliminar caracteres no deseados
    diccionario = json.loads(linea) # Convertir la cadena JSON en un diccionario
    lista.append(diccionario) # Agregar el diccionario a la lista

archivo.close()

df_steam_games = pd.DataFrame(lista)

In [3]:
df_steam_games.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


<br><br>

### 2. Evaluar y trabajar nulos
Se decide eliminar todos los registros con nulos ya que se requiere que los mismos tengan los datos completos para los endpoints (teniendo en cuenta además que muchos item_id del dataset "steam_games" no se encuentran en los item_id de los otros datasets)

<br><br>

In [5]:
df_steam_games.isnull().sum()

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

In [6]:
# borrar registros con nulos

df_steam_games.dropna(inplace=True)

In [7]:
df_steam_games.isnull().sum()

publisher       0
genres          0
app_name        0
title           0
url             0
release_date    0
tags            0
reviews_url     0
specs           0
price           0
early_access    0
id              0
developer       0
dtype: int64

<br><br>

### 3. Evaluar y trabajar duplicados

<br><br>

In [8]:
df_steam_games.duplicated('id').sum()

1

In [9]:
# Para eliminar duplicados se elige solo aquellos con el mismo id ()
df_steam_games.drop_duplicates(subset=["id"], inplace=True)

In [10]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22529 entries, 88310 to 120443
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     22529 non-null  object
 1   genres        22529 non-null  object
 2   app_name      22529 non-null  object
 3   title         22529 non-null  object
 4   url           22529 non-null  object
 5   release_date  22529 non-null  object
 6   tags          22529 non-null  object
 7   reviews_url   22529 non-null  object
 8   specs         22529 non-null  object
 9   price         22529 non-null  object
 10  early_access  22529 non-null  object
 11  id            22529 non-null  object
 12  developer     22529 non-null  object
dtypes: object(13)
memory usage: 2.4+ MB


<br><br>

### 4. Trabajar nombre columnas
* se determina que debían borrarse las siguientes columnas: publisher, url, reviews_url, early_access no se requieren para el proyecto
* se utiliza app_name que es practicamente lo mismo que title solo que contiene menos nulos y menos errores
* se utiliza genres en lugar de tags (podría caer en redundancias y sesgos al aplicar tags sobre el sistema de recomendacion)
* se elimina specs, si bien podría haber sido util para el sistema de recomendación, se observa posible redundancia que puede causar sesgos

<br><br>

In [ ]:
del df_steam_games['publisher']
del df_steam_games['title']
del df_steam_games['url']
del df_steam_games['tags']
del df_steam_games['reviews_url']
del df_steam_games['specs']
del df_steam_games['early_access']

In [12]:
df_steam_games.rename(columns={'id': 'item_id'}, inplace=True)

<br><br>

### 5. Evaluar y trabajar con los tipos de datos de cada campo
Convertir al formato más conveniente
* item_id -- formato int
* price -- formato float
* release_date -- formate int32

<br><br>

In [13]:
df_steam_games['item_id'] = df_steam_games['item_id'].astype(np.int64)

TRABAJAR SOBRE RELEASE_DATE

In [14]:
# Analizar cuantas fechas tiene ingresadas en un formato distinto al mas comun (YYYY-MM-DD)

df_steam_games[~df_steam_games['release_date'].str.match(r"^\d{4}-\d{2}-\d{2}$")].count()

genres          34
app_name        34
release_date    34
price           34
item_id         34
developer       34
dtype: int64

In [15]:
# Analizar aquellas fechas ingresadas en un formato distinto al más comun (YYYY-MM-DD)

df_steam_games[~df_steam_games['release_date'].str.match(r"^\d{4}-\d{2}-\d{2}$")].value_counts('release_date')

# A PARTIR DEL ANALISIS PREVIO SE DETERMINA QUE SE DEBE:
# - Borrar registros que no tienen release_date (aquellos que tienen frases como "soon")
# - El resto dejar unicamente el año expresado

release_date
Oct 2010    3
Jun 2009    2
Nov 2016    2
Apr 2016    1
Sep 2009    1
SOON™       1
SOON        1
Oct 2016    1
Oct 2009    1
Nov 2014    1
May 2015    1
May 2014    1
Mar 2010    1
Jun 2016    1
Jun 2015    1
2018        1
Jul 2017    1
Jul 2016    1
Jul 2014    1
Jul 2010    1
Jan 2017    1
Jan 2015    1
Jan 2010    1
Feb 2013    1
Feb 2011    1
Dec 2012    1
Aug 2015    1
Aug 2014    1
Apr 2017    1
Sep 2014    1
Name: count, dtype: int64

In [16]:
# Borrar aquellos release_date que contengan SOON y coming

soon = r"^SOON"
coming = r"coming|Coming|COMING"
patron = f"({soon})|({coming})"

df_steam_games = df_steam_games[~df_steam_games['release_date'].str.match(patron, case=False)]

In [19]:
# Obtener solo el año de la columna release_date (cargandolo en una nueva columna: release_year)

df_steam_games['release_year'] = df_steam_games['release_date'].str[:4]

df_steam_games['release_year'] = df_steam_games['release_date'].str.extract(r"(\d{4})", expand=False)

df_steam_games['release_year'].fillna(df_steam_games['release_year'].astype(int), inplace=True)

In [20]:
# Borrar la columna "release_date" (reemplazada por release_year)

del df_steam_games['release_date']

In [21]:
# Convertir release_year a formato int32

df_steam_games['release_year']=df_steam_games['release_year'].astype(int)

TRABJAR SOBRE PRICE

In [22]:
# Para ver la lista completa de precios
pd.set_option('display.max_rows', 500)
print(df_steam_games['price'].value_counts())
pd.set_option('display.max_rows', 100)

# Se debe convertir todos los valores no numéricos en "0.0" para luego convertirlos a formato float (ya que todos hacen referencia a free o similar)

price
9.99                             3176
4.99                             3154
0.99                             2178
1.99                             1940
2.99                             1915
19.99                            1402
14.99                            1255
3.99                             1170
6.99                              908
7.99                              723
5.99                              692
Free                              510
Free to Play                      446
29.99                             391
12.99                             361
Free To Play                      337
24.99                             303
8.99                              272
39.99                             228
11.99                             205
49.99                             114
34.99                              85
59.99                              77
16.99                              77
15.99                              71
17.99                              68
10.99 

In [23]:
# Crear una funcion para filtrar valores no numéricos

import re

def filtrar_no_decimales(valor):
    """
    Función para filtrar valores que no sean decimales (maneja tanto cadenas como floats).
    Args:   valor (str o float): Valor a filtrar.
    Returns:   bool: True si el valor no es un decimal, False en caso contrario.
    """
    patron_flotante = r"""^[0-9]+(\.[0-9]{1,2})?$"""

    # Convierte el valor a cadena si es un float
    if isinstance(valor, float):
        valor = str(valor)

    return not re.match(patron_flotante, valor)

In [24]:
# Filtra la columna "price" usando la función filtrar_no_decimales
mascara = df_steam_games["price"].apply(filtrar_no_decimales)

# Convertir los valores no numéricos a "0.0"
df_steam_games.loc[mascara, "price"] = "0.0"

df_steam_games[mascara]

,genres,app_name,price,item_id,developer,release_year
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,0.0,643980,Secret Level SRL,2018
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,0.0,670290,Poolians.com,2017
88370,"[Casual, Indie]",Zen of Sudoku,0.0,4900,Unknown Worlds Entertainment,2006
88501,[Action],Mare Nostrum,0.0,1230,Sandstorm Productions,2008
88558,"[Action, Simulation]",Darkest Hour: Europe '44-'45,0.0,1280,Darklight Games,2009
...,...,...,...,...,...,...
120287,"[Free to Play, Indie, RPG, Strategy]",Aurora Trail,0.0,755150,Screenhit Studios,2017
120303,"[Free to Play, Simulation]",Six Days of Snow,0.0,754810,ebi-hime,2017
120317,"[Action, Casual, Free to Play, Indie]",Bloodworks,0.0,717990,Engin Mercan,2018
120410,"[Action, Free to Play, Massively Multiplayer, ...",忍者村大战2,0.0,754350,杭州分浪网络科技有限公司,2018


In [25]:
# Convertir a formato float la columna price

df_steam_games['price']=df_steam_games["price"].astype(float)

In [26]:
# Convertir a formato int64 la columna "item_id"

df_steam_games['item_id']=df_steam_games["item_id"].astype(np.int64)

In [27]:
# Reset index

df_steam_games = df_steam_games.reset_index(drop=True)

In [28]:
# Reordenar columnas

df_steam_games = df_steam_games.iloc[:,[3,1,5,2,4,0]]

<br><br>

### 6. Aplicar Explode y One-Hot Encoding sobre la columna **genres**

<br><br>

In [144]:
df_steam_games = df_steam_games.explode("genres")

In [147]:
# Convertir los géneros a minúsculas
df_steam_games['genres'] = df_steam_games['genres'].str.lower()

# Reemplazar "&amp;" por "and"
df_steam_games['genres'] = df_steam_games['genres'].str.replace('&amp;', 'and')

df_steam_games.value_counts("genres")

genres
indie                      12691
action                      9428
adventure                   7020
casual                      6282
strategy                    5213
simulation                  4724
rpg                         3809
free to play                1458
early access                1360
sports                      1032
racing                       947
massively multiplayer        813
design and illustration      264
web publishing               195
utilities                    139
animation and modeling        83
education                     53
software training             36
audio production              30
video production              18
photo editing                 11
Name: count, dtype: int64

In [149]:
df_steam_games['developer'] = df_steam_games['developer'].str.lower()

In [150]:
df_steam_games

,item_id,app_name,release_year,price,developer,genres
0,761140,Lost Summoner Kitty,2018,4.99,kotoshiro,action
0,761140,Lost Summoner Kitty,2018,4.99,kotoshiro,casual
0,761140,Lost Summoner Kitty,2018,4.99,kotoshiro,indie
0,761140,Lost Summoner Kitty,2018,4.99,kotoshiro,simulation
0,761140,Lost Summoner Kitty,2018,4.99,kotoshiro,strategy
...,...,...,...,...,...,...
22525,610660,Russian Roads,2018,1.99,laush dmitriy sergeevich,indie
22525,610660,Russian Roads,2018,1.99,laush dmitriy sergeevich,racing
22525,610660,Russian Roads,2018,1.99,laush dmitriy sergeevich,simulation
22526,658870,EXIT 2 - Directions,2017,4.99,"xropi,stev3ns",casual


In [151]:
# EXPORTAR EL DATAFRAME EN FORMATO PARQUET

df_steam_games.to_parquet("df_steam_games.parquet")

In [146]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55606 entries, 0 to 22526
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   item_id       55606 non-null  int64  
 1   app_name      55606 non-null  object 
 2   release_year  55606 non-null  int32  
 3   price         55606 non-null  float64
 4   developer     55606 non-null  object 
 5   genres        55606 non-null  object 
dtypes: float64(1), int32(1), int64(1), object(3)
memory usage: 2.8+ MB


In [8]:
# Aplicar one-hot encoding a la columna 'genres'
df_steam_games_dummies = pd.get_dummies(df_steam_games, columns=['genres'])

# Agrupar por las columnas originales y sumar los valores de las columnas dummies
df_steam_games_dummies = df_steam_games_dummies.groupby(['item_id', 'app_name', 'release_year', 'price', 'developer'], as_index=False).sum()


In [9]:
df_steam_games_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22527 entries, 0 to 22526
Data columns (total 26 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   item_id                           22527 non-null  int64  
 1   app_name                          22527 non-null  object 
 2   release_year                      22527 non-null  int32  
 3   price                             22527 non-null  float64
 4   developer                         22527 non-null  object 
 5   genres_Action                     22527 non-null  int64  
 6   genres_Adventure                  22527 non-null  int64  
 7   genres_Animation &amp; Modeling   22527 non-null  int64  
 8   genres_Audio Production           22527 non-null  int64  
 9   genres_Casual                     22527 non-null  int64  
 10  genres_Design &amp; Illustration  22527 non-null  int64  
 11  genres_Early Access               22527 non-null  int64  
 12  genr

In [10]:
df_steam_games_dummies.rename(columns={
    'genres_Action':'action',
    'genres_Adventure':'adventure',
    'genres_Animation &amp; Modeling':'animation_modeling',
    'genres_Audio Production':'audio_production',
    "genres_Casual":"casual",
    "genres_Design &amp; Illustration":"design_illustration",
    "genres_Early Access":"early_access",
    "genres_Education":"education",
    "genres_Free to Play":"free_play",
    "genres_Indie":"indie",
    "genres_Masssively Multiplayer":"massively_multiplayer",
    "genres_Photo Editing":"photo_editing",
    "genres_RPG":"rpg",
    "genres_Racing":"racing",
    "genres_Simulation":"simulation",
    "genres_Software Training":"software_training",
    "genres_Sports":"sports",
    "genres_Strategy":"strategy",
    "genres_Utilities":"utilities",
    "genres_Video Production":"video_production",
    "genres_Web Publishing":"web_publishing"},
      inplace=True)

In [11]:
df_steam_games_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22527 entries, 0 to 22526
Data columns (total 26 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   item_id                       22527 non-null  int64  
 1   app_name                      22527 non-null  object 
 2   release_year                  22527 non-null  int32  
 3   price                         22527 non-null  float64
 4   developer                     22527 non-null  object 
 5   action                        22527 non-null  int64  
 6   adventure                     22527 non-null  int64  
 7   animation_modeling            22527 non-null  int64  
 8   audio_production              22527 non-null  int64  
 9   casual                        22527 non-null  int64  
 10  design_illustration           22527 non-null  int64  
 11  early_access                  22527 non-null  int64  
 12  education                     22527 non-null  int64  
 13  f

<br><br>

## 

<br><br>

## PARTE 2 - DATASET USER_REVIEWS
### OBJETIVOS
* Cargar el dataset australian_user_reviews.json
* Crear la columna de 'analisis de sentimientos' basado en los reviews con NLTK
* Realizar análisis básicos y transformaciones según se requiera para el MVP
* Exportar dataframes en formato parquet

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

### 1. Cargar el dataset 'australian_user_reviews.json'

In [3]:
ruta = 'datasets/australian_user_reviews.json'

with open(ruta, 'r', encoding='utf-8') as f:
    data = f.readlines()

# convertir las lineas en registros
records = [eval(line.strip()) for line in data]

# Creamos el DataFrame a partir de los records o registros
df_UserReviews= pd.DataFrame(records)

In [ ]:
df_UserReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [ ]:
# Evaluar los datos anidados de la columna reviews
df_UserReviews['reviews'][0]

# se observan datos anidados (solo se requieren "posted", "item_id", "recommend" y "review")

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

Trabajos iniciales a partir de un análisis básico del dataframe:
* Borrar "user_url" -- no tiene relevancia para el proyecto (podría servir para buscar posibles datos faltantes o con errores en los registros, no es el caso).
* Borrar duplicados
* Desanidar los datos en la columna "reviews" con explode, y crear 4 columnas para incorporar "posted", "item_id", "recommend" y "review"; el resto no son datos relevantes para el proyecto

### 2. Borrar la columna "user_url" y los duplicados

In [ ]:
# contar duplicados

# df_UserReviews_original.dropna().duplicated().sum()
df_UserReviews.duplicated('reviews').sum()

340

In [ ]:
df_UserReviews.duplicated('user_id').sum()

# probablemente hay 26 duplicados mas en la columna reviews porque hay 27 nulos (que no los toma como nulos porque son listas vacías [])
# y solo cuenta 26 porque deja 1 como valor original

314

In [ ]:
# borrar los 314 users_id duplicados

# df_UserReviews_original.dropna().drop_duplicates()
df_UserReviews.drop_duplicates('user_id',inplace=True)

In [ ]:
# borrar user_url

del df_UserReviews['user_url']


### 3. Aplicar **explode** a la columna reviews con datos anidados

In [ ]:
df_UserReviews = df_UserReviews.explode('reviews')

In [ ]:
df_UserReviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58458 entries, 0 to 25798
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  58458 non-null  object
 1   reviews  58430 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


In [ ]:
df_UserReviews.head()

,user_id,reviews
0,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,76561197970982479,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,76561197970982479,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."


### 4. Evaluar nulos

In [ ]:
# Contar la cantidad de nulos por columna
df_UserReviews.isna().sum()

user_id     0
reviews    28
dtype: int64

In [ ]:
# Mostrar unicamente con los registros que tienen nulos en reviews (a modo de chequeo)
df_UserReviews[df_UserReviews['reviews'].isnull()].head(10)


,user_id,reviews
62,gdxsd,NaN
83,76561198094224872,NaN
1047,76561198021575394,NaN
3954,cmuir37,NaN
5394,Jaysteeny,NaN
6135,ML8989,NaN
7583,76561198079215291,NaN
7952,76561198079342142,NaN
9894,76561198061996985,NaN
10381,76561198108286351,NaN


### 5. Borrar los registros con nulos (NaN) en la columna reviews

In [ ]:
# Borrar los registros con nulos en reviews
df_UserReviews = df_UserReviews.dropna(subset='reviews')

In [ ]:
df_UserReviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58430 entries, 0 to 25798
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  58430 non-null  object
 1   reviews  58430 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


### 6. Reindexar los registros para trabajarlos con mayor comodidad

In [ ]:
df_UserReviews.reset_index(drop=True, inplace=True)

In [ ]:
df_UserReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58430 entries, 0 to 58429
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  58430 non-null  object
 1   reviews  58430 non-null  object
dtypes: object(2)
memory usage: 913.1+ KB


In [ ]:
len(df_UserReviews)

58430

In [ ]:
len(df_UserReviews['user_id'].unique())

25457

### 7. Crear las columnas "posted", "item_id", "recommend" y "review" (con los datos correspondientes) y borrar "reviews"

In [ ]:
df_UserReviews.iloc[:,1]

0        {'funny': '', 'posted': 'Posted November 5, 20...
1        {'funny': '', 'posted': 'Posted July 15, 2011....
2        {'funny': '', 'posted': 'Posted April 21, 2011...
3        {'funny': '', 'posted': 'Posted June 24, 2014....
4        {'funny': '', 'posted': 'Posted September 8, 2...
                               ...                        
58425    {'funny': '', 'posted': 'Posted July 10.', 'la...
58426    {'funny': '', 'posted': 'Posted July 8.', 'las...
58427    {'funny': '1 person found this review funny', ...
58428    {'funny': '', 'posted': 'Posted July 20.', 'la...
58429    {'funny': '', 'posted': 'Posted July 2.', 'las...
Name: reviews, Length: 58430, dtype: object

In [ ]:
# Crear posted e incorporar los datos correspondientes

for i, dic in enumerate(df_UserReviews.loc[:,'reviews']):
    if "posted" in dic:
        df_UserReviews.loc[i, "posted"] = dic["posted"]
    else:
        # Asignar un valor predeterminado o omitir el procesamiento
        df_UserReviews.loc[i, "posted"] = None  # Valor predeterminado
        continue  # Omitir el procesamiento

C:\Users\Intel\AppData\Local\Temp\ipykernel_8000\3897464620.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_UserReviews.loc[i, "posted"] = dic["posted"]


In [ ]:
df_UserReviews.head()

,user_id,reviews,posted
0,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20...","Posted November 5, 2011."
1,76561197970982479,"{'funny': '', 'posted': 'Posted July 15, 2011....","Posted July 15, 2011."
2,76561197970982479,"{'funny': '', 'posted': 'Posted April 21, 2011...","Posted April 21, 2011."
3,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","Posted June 24, 2014."
4,js41637,"{'funny': '', 'posted': 'Posted September 8, 2...","Posted September 8, 2013."


In [ ]:
df_UserReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58430 entries, 0 to 58429
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  58430 non-null  object
 1   reviews  58430 non-null  object
 2   posted   58430 non-null  object
dtypes: object(3)
memory usage: 1.3+ MB


In [ ]:
# Crear item_id e incorporar los datos correspondientes

for i, dic in enumerate(df_UserReviews.iloc[:,1]):
    if "item_id" in dic:
        df_UserReviews.loc[i, "item_id"] = str(dic["item_id"])
    else:
        # Asignar un valor predeterminado o omitir el procesamiento
        df_UserReviews.loc[i, "item_id"] = None  # Valor predeterminado
        continue  # Omitir el procesamiento

C:\Users\Intel\AppData\Local\Temp\ipykernel_8000\2215573292.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_UserReviews.loc[i, "item_id"] = str(dic["item_id"])


In [ ]:
df_UserReviews.head()

,user_id,reviews,posted,item_id
0,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20...","Posted November 5, 2011.",1250
1,76561197970982479,"{'funny': '', 'posted': 'Posted July 15, 2011....","Posted July 15, 2011.",22200
2,76561197970982479,"{'funny': '', 'posted': 'Posted April 21, 2011...","Posted April 21, 2011.",43110
3,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","Posted June 24, 2014.",251610
4,js41637,"{'funny': '', 'posted': 'Posted September 8, 2...","Posted September 8, 2013.",227300


In [ ]:
df_UserReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58430 entries, 0 to 58429
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  58430 non-null  object
 1   reviews  58430 non-null  object
 2   posted   58430 non-null  object
 3   item_id  58430 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [ ]:
# Crear recommend e incorporar los datos correspondientes

for i, dic in enumerate(df_UserReviews.iloc[:,1]):
    if "recommend" in dic:
        df_UserReviews.loc[i, "recommend"] = str(dic["recommend"])
    else:
        # Asignar un valor predeterminado o omitir el procesamiento
        df_UserReviews.loc[i, "recommend"] = None  # Valor predeterminado
        continue  # Omitir el procesamiento

C:\Users\Intel\AppData\Local\Temp\ipykernel_8000\3523201857.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_UserReviews.loc[i, "recommend"] = str(dic["recommend"])


In [ ]:
df_UserReviews.head()

,user_id,reviews,posted,item_id,recommend
0,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20...","Posted November 5, 2011.",1250,True
1,76561197970982479,"{'funny': '', 'posted': 'Posted July 15, 2011....","Posted July 15, 2011.",22200,True
2,76561197970982479,"{'funny': '', 'posted': 'Posted April 21, 2011...","Posted April 21, 2011.",43110,True
3,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","Posted June 24, 2014.",251610,True
4,js41637,"{'funny': '', 'posted': 'Posted September 8, 2...","Posted September 8, 2013.",227300,True


In [ ]:
df_UserReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58430 entries, 0 to 58429
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    58430 non-null  object
 1   reviews    58430 non-null  object
 2   posted     58430 non-null  object
 3   item_id    58430 non-null  object
 4   recommend  58430 non-null  object
dtypes: object(5)
memory usage: 2.2+ MB


In [ ]:
# Crear review e incorporar los datos correspondientes

for i, dic in enumerate(df_UserReviews.iloc[:,1]):
    if "review" in dic:
        df_UserReviews.loc[i, "review"] = str(dic["review"])
    else:
        # Asignar un valor predeterminado o omitir el procesamiento
        df_UserReviews.loc[i, "review"] = None  # Valor predeterminado
        continue  # Omitir el procesamiento

C:\Users\Intel\AppData\Local\Temp\ipykernel_8000\1366936764.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_UserReviews.loc[i, "review"] = str(dic["review"])


In [ ]:
df_UserReviews.head()

,user_id,reviews,posted,item_id,recommend,review
0,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20...","Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"{'funny': '', 'posted': 'Posted July 15, 2011....","Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"{'funny': '', 'posted': 'Posted April 21, 2011...","Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"{'funny': '', 'posted': 'Posted September 8, 2...","Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...


In [ ]:
df_UserReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58430 entries, 0 to 58429
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    58430 non-null  object
 1   reviews    58430 non-null  object
 2   posted     58430 non-null  object
 3   item_id    58430 non-null  object
 4   recommend  58430 non-null  object
 5   review     58430 non-null  object
dtypes: object(6)
memory usage: 2.7+ MB


In [ ]:
# Borrar la columna 'reviews'

del df_UserReviews['reviews']

In [ ]:
df_UserReviews.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...


In [ ]:
df_UserReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58430 entries, 0 to 58429
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    58430 non-null  object
 1   posted     58430 non-null  object
 2   item_id    58430 non-null  object
 3   recommend  58430 non-null  object
 4   review     58430 non-null  object
dtypes: object(5)
memory usage: 2.2+ MB


### 8. Dejar solo el año en la columna Posted (con respecto a fechas es lo único que se requiere en el proyecto)

In [ ]:
df_UserReviews['posted']=df_UserReviews['posted'].str.strip('.').str.strip('Posted ')

C:\Users\Intel\AppData\Local\Temp\ipykernel_8000\3258059106.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_UserReviews['posted']=df_UserReviews['posted'].str.strip('.').str.strip('Posted ')


In [ ]:
df_UserReviews.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,"November 5, 2011",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"July 15, 2011",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"April 21, 2011",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"June 24, 2014",251610,True,I know what you think when you see this title ...
4,js41637,"September 8, 2013",227300,True,For a simple (it's actually not all that simpl...


In [ ]:
df_UserReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58430 entries, 0 to 58429
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    58430 non-null  object
 1   posted     58430 non-null  object
 2   item_id    58430 non-null  object
 3   recommend  58430 non-null  object
 4   review     58430 non-null  object
dtypes: object(5)
memory usage: 2.2+ MB


In [ ]:
df_UserReviews['posted'] = df_UserReviews['posted'].str.split(', ').str.get(1)

C:\Users\Intel\AppData\Local\Temp\ipykernel_8000\3664659044.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_UserReviews['posted'] = df_UserReviews['posted'].str.split(', ').str.get(1)


In [ ]:
df_UserReviews.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,2011,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,2011,22200,True,It's unique and worth a playthrough.
2,76561197970982479,2011,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,2014,251610,True,I know what you think when you see this title ...
4,js41637,2013,227300,True,For a simple (it's actually not all that simpl...


In [ ]:
df_UserReviews.info()

# SE OBSERVA MÁS DE 10 MIL DATOS NULOS EN 'POSTED'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58430 entries, 0 to 58429
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    58430 non-null  object
 1   posted     48498 non-null  object
 2   item_id    58430 non-null  object
 3   recommend  58430 non-null  object
 4   review     58430 non-null  object
dtypes: object(5)
memory usage: 2.2+ MB


Opciones valoradas bajo distintos criterios para trabajar los datos nulos de la columna Posted:
* Dejarlos nulos
* Borrarlos
* Imputarlos

En el proyecto, solo un endpoint requiere del 'posted', otros que requieren "user_reviews" no necesita consultar los años. Por lo tanto, el criterio que usaré es dejar los nulos, pero los convertiré en 0 (para convertirlo en formato int32) y para el endpoint que los requiere simplemente filtrarlos, para que no intente contabilizar aquellos nulos.. Los endpoints que manejan datos de user_reviews son los siguientes:

def userdata( User_id : str )  -- utiliza recommend (no necesito filtrar)

def best_developer_year( año : int )  -- utiliza posted (debo filtrarlos)

def developer_reviews_analysis( desarrolladora : str )  -- utiliza sentiment_analysis (no necesito filtrar)

### 9. Hacer análisis de sentimiento a los reviews con nltk
Crear columna sentiment_analysis con valores:
* 0 para comentarios negativos
* 1 para comentarios neutros
* 2 para comentarios positivos

Usaré como criterio en polarity_scores(texto)["compound"]:
- Mayor a 0.4 tomar como comentarios positivos
- Entre -0.4 y 0.4 tomar como comentarios neutros
- Menor a -0.4 tomar como comentarios negativos

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
sia = SentimentIntensityAnalyzer()

In [ ]:
def analizar_sentimiento(texto):
	sia = SentimentIntensityAnalyzer()
	sentimiento = sia.polarity_scores(texto)["compound"]
	if sentimiento > 0.4:
		return 2
	elif sentimiento < -0.4:
		return 0
	else:
		return 1

In [ ]:
df_UserReviews['sentiment_analysis'] = df_UserReviews['review'].apply(analizar_sentimiento)

df_UserReviews

C:\Users\Intel\AppData\Local\Temp\ipykernel_8000\110911918.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_UserReviews['sentiment_analysis'] = df_UserReviews['review'].apply(analizar_sentimiento)


,user_id,posted,item_id,recommend,review,sentiment_analysis
0,76561197970982479,2011,1250,True,Simple yet with great replayability. In my opi...,2
1,76561197970982479,2011,22200,True,It's unique and worth a playthrough.,1
2,76561197970982479,2011,43110,True,Great atmosphere. The gunplay can be a bit chu...,2
3,js41637,2014,251610,True,I know what you think when you see this title ...,2
4,js41637,2013,227300,True,For a simple (it's actually not all that simpl...,2
...,...,...,...,...,...,...
58425,76561198312638244,NaN,70,True,a must have classic from steam definitely wort...,2
58426,76561198312638244,NaN,362890,True,this game is a perfect remake of the original ...,2
58427,LydiaMorley,NaN,273110,True,had so much fun plaing this and collecting res...,2
58428,LydiaMorley,NaN,730,True,:D,2


In [ ]:
df_UserReviews['sentiment_analysis'].value_counts()

sentiment_analysis
2    31790
1    20480
0     6160
Name: count, dtype: int64

In [ ]:
# Leer algunos reviews clasificados para corroborar que funciona correctamente

contador = 0
for i,n in enumerate(df_UserReviews['sentiment_analysis']):
    if contador < 10:
        if n == 1:
            print(df_UserReviews['review'][i])
            contador += 1

It's unique and worth a playthrough.
A suitably punishing roguelike platformer.  Winning feels good.  Progressive unlocks mean a good slog ending in failure doesn't feel like a waste.
"Run for fun? What the hell kind of fun is that?"
Random drops and random quests, with stat points.  Animation style reminiscent of the era before the Voodoo card.
Git gud
Hold shift to win, Hold CTRL to lose.
This game is Marvellous.
It reminds me of that TV Show called "The Walking Dead".
If you like defending dungeons then I suppose this game would suit your needs.
You don't have to get this game. It's not like it's the greatest FPS of all time or anything. But have a think about it.


##### ES NECESARIO AJUSTAR PARAMETROS O USAR OTRA HERRAMIENTA PARA EL ANALISIS DE SENTIMIENTO
Se tuvo muy mala respuesta de la herramienta de analisis de sentimiento usada (analizando distintas muestras se obtiene una precisión aprox. del 30%)

In [ ]:
df_UserReviews

,user_id,posted,item_id,recommend,review,sentiment_analysis
0,76561197970982479,2011,1250,True,Simple yet with great replayability. In my opi...,2
1,76561197970982479,2011,22200,True,It's unique and worth a playthrough.,1
2,76561197970982479,2011,43110,True,Great atmosphere. The gunplay can be a bit chu...,2
3,js41637,2014,251610,True,I know what you think when you see this title ...,2
4,js41637,2013,227300,True,For a simple (it's actually not all that simpl...,2
...,...,...,...,...,...,...
58425,76561198312638244,NaN,70,True,a must have classic from steam definitely wort...,2
58426,76561198312638244,NaN,362890,True,this game is a perfect remake of the original ...,2
58427,LydiaMorley,NaN,273110,True,had so much fun plaing this and collecting res...,2
58428,LydiaMorley,NaN,730,True,:D,2


In [ ]:
# Crear un dataframe nuevo: df_UserReviews_final, dejando df_UserReviews_exploded con los reviews por si requiere hacer consultas

df_UserReviews_final = df_UserReviews
del df_UserReviews_final['review']

In [ ]:
df_UserReviews_final

,user_id,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,2011,1250,True,2
1,76561197970982479,2011,22200,True,1
2,76561197970982479,2011,43110,True,2
3,js41637,2014,251610,True,2
4,js41637,2013,227300,True,2
...,...,...,...,...,...
58425,76561198312638244,NaN,70,True,2
58426,76561198312638244,NaN,362890,True,2
58427,LydiaMorley,NaN,273110,True,2
58428,LydiaMorley,NaN,730,True,2


In [ ]:
df_UserReviews_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58430 entries, 0 to 58429
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             58430 non-null  object
 1   posted              48498 non-null  object
 2   item_id             58430 non-null  object
 3   recommend           58430 non-null  object
 4   sentiment_analysis  58430 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.2+ MB


### 10. Convertir a los formatos correspondientes las columnas:
* posted: int32  -- a los nulos convertirlos en 0 (y filtrarlos cuando deba utilizar la columna posted)
* recommend: 0 y 1 (int)
* user_id: probablemente conviene evaluar si usar steam_id de la otra tabla (y así podría pasarlo a int)
* item_id: int

In [ ]:
# Convertir a en 0: False y 1: True la columna recommend

df_UserReviews_final['recommend'] = df_UserReviews_final['recommend'].replace({'True': 1, 'False': 0})

C:\Users\Intel\AppData\Local\Temp\ipykernel_8000\340443496.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_UserReviews_final['recommend'] = df_UserReviews_final['recommend'].replace({'True': 1, 'False': 0})


In [ ]:
# Otorgar el valor 0 a los registros con "posted" nulos

df_UserReviews_final['posted'] = df_UserReviews_final['posted'].fillna(0)

C:\Users\Intel\AppData\Local\Temp\ipykernel_8000\1264704029.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_UserReviews_final['posted'] = df_UserReviews_final['posted'].fillna(0)


In [ ]:
# Convertir a formato int la columna posted
df_UserReviews_final['posted'] = df_UserReviews_final['posted'].astype(int)

C:\Users\Intel\AppData\Local\Temp\ipykernel_8000\2322815019.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_UserReviews_final['posted'] = df_UserReviews_final['posted'].astype(int)


In [ ]:
df_UserReviews_final

,user_id,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,2011,1250,1,2
1,76561197970982479,2011,22200,1,1
2,76561197970982479,2011,43110,1,2
3,js41637,2014,251610,1,2
4,js41637,2013,227300,1,2
...,...,...,...,...,...
58425,76561198312638244,0,70,1,2
58426,76561198312638244,0,362890,1,2
58427,LydiaMorley,0,273110,1,2
58428,LydiaMorley,0,730,1,2


In [ ]:
df_UserReviews_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58430 entries, 0 to 58429
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             58430 non-null  object
 1   posted              58430 non-null  int32 
 2   item_id             58430 non-null  object
 3   recommend           58430 non-null  int64 
 4   sentiment_analysis  58430 non-null  int64 
dtypes: int32(1), int64(2), object(2)
memory usage: 2.0+ MB


In [ ]:
df_UserReviews_final['item_id'] = df_UserReviews_final['item_id'].astype(int)

C:\Users\Intel\AppData\Local\Temp\ipykernel_8000\3586761343.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_UserReviews_final['item_id'] = df_UserReviews_final['item_id'].astype(int)


### 11. Exportar a formato parquet

In [ ]:
df_UserReviews_final.to_parquet('df_user_reviews.parquet')

<br><br><br>

# PARTE 3 - DATASET USERS_ITEMS
### OBJETIVOS
* Cargar el dataset australian_users_items.json
* Realizar transformaciones y limpieza segun se requiera
* Exportar el dataframe final en formato parquet

<br>

### 1. Cargar el dataset 'australian_users_items.json'

In [ ]:
# Nota: el archivo 'australian_users_items.json' no se encuentra en GitHub. Se cuenta solo con users_items.parquet (con el ETL realizado)

ruta = 'datasets/australian_users_items.json'

with open(ruta, 'r', encoding='utf-8') as f:
    data = f.readlines()

# convertir las lineas en registros
records = [eval(line.strip()) for line in data]

# Creamos el DataFrame a partir de los records o registros
df_UsersItems_original= pd.DataFrame(records)

In [ ]:
df_UsersItems_original

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [ ]:
df_UsersItems_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


### 2. Borrar nulos y duplicados, desanidar columna "items" con exploded

In [ ]:
# contar duplicados

# df_UserReviews_original.dropna().duplicated().sum()
df_UsersItems_original.duplicated('items').sum()

19408

In [ ]:
df_UsersItems_original.duplicated('user_id').sum()

684

In [ ]:
df_UsersItems_original.duplicated('steam_id').sum()

685

In [ ]:
# borrar los 314 users_id duplicados

# df_UserReviews_original.dropna().drop_duplicates()
df_UsersItems_original.drop_duplicates('steam_id',inplace=True)

In [ ]:
df_UsersItems_exploded = df_UsersItems_original.explode('items')

In [ ]:
df_UsersItems_exploded

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."
...,...,...,...,...,...
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '373330', 'item_name': 'All Is Dus..."
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '388490', 'item_name': 'One Way To..."
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '521570', 'item_name': 'You Have 1..."
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '519140', 'item_name': 'Minds Eyes..."


In [ ]:
df_UsersItems_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5110795 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   user_id      object
 1   items_count  int64 
 2   steam_id     object
 3   user_url     object
 4   items        object
dtypes: int64(1), object(4)
memory usage: 234.0+ MB


In [ ]:
# Evaluar los datos anidados de la columna reviews
contador = 0
for i,n in enumerate(df_UsersItems_exploded['items']):
    if contador < 5:
        print(n)
        contador +=1

# se observan datos anidados (solo se requieren 

{'item_id': '10', 'item_name': 'Counter-Strike', 'playtime_forever': 6, 'playtime_2weeks': 0}
{'item_id': '20', 'item_name': 'Team Fortress Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}
{'item_id': '30', 'item_name': 'Day of Defeat', 'playtime_forever': 7, 'playtime_2weeks': 0}
{'item_id': '40', 'item_name': 'Deathmatch Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}
{'item_id': '50', 'item_name': 'Half-Life: Opposing Force', 'playtime_forever': 0, 'playtime_2weeks': 0}


Columnas que considero utiles para el proyecto:
* user_id
>- item_id
>- item_name (que estimo es lo mismo que app_name o title en steam_games.json)
>- playtime_forever

### 3. Borrar columnas "user_url", items_count, steam_id

In [ ]:
del df_UsersItems_exploded['user_url']
del df_UsersItems_exploded['items_count']

In [ ]:
df_UsersItems_exploded

,user_id,steam_id,items
0,76561197970982479,76561197970982479,"{'item_id': '10', 'item_name': 'Counter-Strike..."
0,76561197970982479,76561197970982479,"{'item_id': '20', 'item_name': 'Team Fortress ..."
0,76561197970982479,76561197970982479,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
0,76561197970982479,76561197970982479,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
0,76561197970982479,76561197970982479,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."
...,...,...,...
88308,76561198329548331,76561198329548331,"{'item_id': '373330', 'item_name': 'All Is Dus..."
88308,76561198329548331,76561198329548331,"{'item_id': '388490', 'item_name': 'One Way To..."
88308,76561198329548331,76561198329548331,"{'item_id': '521570', 'item_name': 'You Have 1..."
88308,76561198329548331,76561198329548331,"{'item_id': '519140', 'item_name': 'Minds Eyes..."


In [ ]:
df_UsersItems_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5110795 entries, 0 to 88309
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   object
 1   steam_id  object
 2   items     object
dtypes: object(3)
memory usage: 156.0+ MB


### 4. Evaluar nulos

In [ ]:
df_UsersItems_exploded.isnull().sum()

user_id         0
steam_id        0
items       16713
dtype: int64

In [ ]:
# se decide borrar los registros con datos nulos
df_UsersItems_exploded.dropna(inplace=True)

In [ ]:
df_UsersItems_exploded.isnull().sum()


user_id     0
steam_id    0
items       0
dtype: int64

In [ ]:
df_UsersItems_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5094082 entries, 0 to 88308
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   object
 1   steam_id  object
 2   items     object
dtypes: object(3)
memory usage: 155.5+ MB


### 5. Reindexar los registros para trabajarlos con mayor comodidad

In [ ]:
df_UsersItems_exploded.reset_index(drop=True, inplace=True)

In [ ]:
df_UsersItems_exploded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094082 entries, 0 to 5094081
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   object
 1   steam_id  object
 2   items     object
dtypes: object(3)
memory usage: 116.6+ MB


### 6. Crear las columnas item_id, item_name, playtime_forever (y borrar items)

In [ ]:
# Crear item_id e incorporar los datos correspondientes

for i, dic in enumerate(df_UsersItems_exploded.loc[:,'items']):
    if "item_id" in dic:
        df_UsersItems_exploded.loc[i, "item_id"] = dic["item_id"]
    else:
        # Asignar un valor predeterminado o omitir el procesamiento
        df_UsersItems_exploded.loc[i, "item_id"] = None  # Valor predeterminado
        continue  # Omitir el procesamiento

In [ ]:
df_UsersItems_exploded

,user_id,steam_id,items,item_id
0,76561197970982479,76561197970982479,"{'item_id': '10', 'item_name': 'Counter-Strike...",10
1,76561197970982479,76561197970982479,"{'item_id': '20', 'item_name': 'Team Fortress ...",20
2,76561197970982479,76561197970982479,"{'item_id': '30', 'item_name': 'Day of Defeat'...",30
3,76561197970982479,76561197970982479,"{'item_id': '40', 'item_name': 'Deathmatch Cla...",40
4,76561197970982479,76561197970982479,"{'item_id': '50', 'item_name': 'Half-Life: Opp...",50
...,...,...,...,...
5094077,76561198329548331,76561198329548331,"{'item_id': '346330', 'item_name': 'BrainBread...",346330
5094078,76561198329548331,76561198329548331,"{'item_id': '373330', 'item_name': 'All Is Dus...",373330
5094079,76561198329548331,76561198329548331,"{'item_id': '388490', 'item_name': 'One Way To...",388490
5094080,76561198329548331,76561198329548331,"{'item_id': '521570', 'item_name': 'You Have 1...",521570


In [ ]:
df_UsersItems_exploded.info()

# se puede 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094082 entries, 0 to 5094081
Data columns (total 4 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   object
 1   steam_id  object
 2   items     object
 3   item_id   object
dtypes: object(4)
memory usage: 155.5+ MB


In [ ]:
df_UsersItems_exploded.isnull().sum()

user_id     0
steam_id    0
items       0
item_id     0
dtype: int64

In [ ]:
# Crear item_name e incorporar los datos correspondientes

for i, dic in enumerate(df_UsersItems_exploded.loc[:,'items']):
    if "item_name" in dic:
        df_UsersItems_exploded.loc[i, "item_name"] = dic["item_name"]
    else:
        # Asignar un valor predeterminado o omitir el procesamiento
        df_UsersItems_exploded.loc[i, "item_name"] = None  # Valor predeterminado
        continue  # Omitir el procesamiento

In [ ]:
df_UsersItems_exploded.head()

,user_id,steam_id,items,item_id,item_name
0,76561197970982479,76561197970982479,"{'item_id': '10', 'item_name': 'Counter-Strike...",10,Counter-Strike
1,76561197970982479,76561197970982479,"{'item_id': '20', 'item_name': 'Team Fortress ...",20,Team Fortress Classic
2,76561197970982479,76561197970982479,"{'item_id': '30', 'item_name': 'Day of Defeat'...",30,Day of Defeat
3,76561197970982479,76561197970982479,"{'item_id': '40', 'item_name': 'Deathmatch Cla...",40,Deathmatch Classic
4,76561197970982479,76561197970982479,"{'item_id': '50', 'item_name': 'Half-Life: Opp...",50,Half-Life: Opposing Force


In [ ]:
df_UsersItems_exploded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094082 entries, 0 to 5094081
Data columns (total 5 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   user_id    object
 1   steam_id   object
 2   items      object
 3   item_id    object
 4   item_name  object
dtypes: object(5)
memory usage: 194.3+ MB


In [ ]:
# Crear playtime_forever e incorporar los datos correspondientes

for i, dic in enumerate(df_UsersItems_exploded.loc[:,'items']):
    if "playtime_forever" in dic:
        df_UsersItems_exploded.loc[i, "playtime_forever"] = dic["playtime_forever"]
    else:
        # Asignar un valor predeterminado o omitir el procesamiento
        df_UsersItems_exploded.loc[i, "playtime_forever"] = None  # Valor predeterminado
        continue  # Omitir el procesamiento

In [ ]:
df_UsersItems_exploded.head()

,user_id,steam_id,items,item_id,item_name,playtime_forever
0,76561197970982479,76561197970982479,"{'item_id': '10', 'item_name': 'Counter-Strike...",10,Counter-Strike,6.0
1,76561197970982479,76561197970982479,"{'item_id': '20', 'item_name': 'Team Fortress ...",20,Team Fortress Classic,0.0
2,76561197970982479,76561197970982479,"{'item_id': '30', 'item_name': 'Day of Defeat'...",30,Day of Defeat,7.0
3,76561197970982479,76561197970982479,"{'item_id': '40', 'item_name': 'Deathmatch Cla...",40,Deathmatch Classic,0.0
4,76561197970982479,76561197970982479,"{'item_id': '50', 'item_name': 'Half-Life: Opp...",50,Half-Life: Opposing Force,0.0


In [ ]:
df_UsersItems_exploded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094082 entries, 0 to 5094081
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   steam_id          object 
 2   items             object 
 3   item_id           object 
 4   item_name         object 
 5   playtime_forever  float64
dtypes: float64(1), object(5)
memory usage: 233.2+ MB


In [ ]:
df_UsersItems_exploded.isnull().sum()

user_id             0
steam_id            0
items               0
item_id             0
item_name           0
playtime_forever    0
dtype: int64

In [ ]:
del df_UsersItems_exploded['items']
del df_UsersItems_exploded['steam_id']

In [ ]:
df_UsersItems_exploded['item_id'] = df_UsersItems_exploded['item_id'].astype(np.int64)

### 7. Exportar a formato parquet

In [ ]:
df_UsersItems_exploded.to_parquet('users_items.parquet')

<br><br>

# PARTE 4 - CREAR DATAFRAMES PARA LOS ENDPOINTS

<br><br>

In [30]:
# Importar bibliotecas (intentar lo minimo indispensable)

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings("ignore")

In [153]:
# Importar los datasets limpios (en formato parquet)

df_g = pd.read_parquet("datasets\\df_steam_games.parquet")
df_r = pd.read_parquet("datasets\\user_reviews.parquet")
df_i = pd.read_parquet("datasets\\users_items.parquet")

<br>
<br>

## 1)   Def_developer

<br>

#### Cantidad items y porcentaje contenido free por año, por empresa desarrolladora

<br>
<br>

Se obtienen las columnas necesarias del dataframe de steam_games

In [155]:
df_filtrado = df_g[['price', 'release_year', 'developer', 'item_id']]
# se eliminan los duplicados
df_developer = df_filtrado.drop_duplicates()
df_developer

,price,release_year,developer,item_id
0,4.99,2018,kotoshiro,761140
1,0.00,2018,secret level srl,643980
2,0.00,2017,poolians.com,670290
3,0.99,2017,彼岸领域,767400
4,3.99,2018,trickjump games ltd,772540
...,...,...,...,...
22522,1.99,2018,bidoniera games,745400
22523,1.99,2018,"nikita ""ghost_rus""",773640
22524,4.99,2018,sacada,733530
22525,1.99,2018,laush dmitriy sergeevich,610660


<br>
<br>

#### Se guarda como parquet

In [156]:
# Convierte el DataFrame a un objeto Table de PyArrow
table = pa.Table.from_pandas(df_developer)

# Define las opciones de compresión con Snappy
compression = 'None'

# Guarda el objeto Table en formato Parquet con compresión Snappy
pq.write_table(table, "datasets\\def_developer.parquet", compression=compression)

<br>
<br>

## 2) Def user_data

<br>

#### Cantidad de dinero gastado por el usuario, el porcentaje de recomendación (reviews.recommend) y cantidad de items.

<br>
<br>

Se obtienen las columnas 'price' e 'item_id' del Dataframe de steam_games, descartando los valores duplicados

In [157]:
dinero_gastado = df_g[['price', 'item_id']]
dinero_gastado = dinero_gastado.drop_duplicates(subset='item_id', keep='first')
dinero_gastado

,price,item_id
0,4.99,761140
1,0.00,643980
2,0.00,670290
3,0.99,767400
4,3.99,772540
...,...,...
22522,1.99,745400
22523,1.99,773640
22524,4.99,733530
22525,1.99,610660


<br>
<br>

Se obtienen las columnas 'item_id', 'item_count' y 'user_id' del Dataframe users_items

In [158]:
# Llevar a minuscula item_name
df_i['item_name'] = df_i['item_name'].str.lower()
df_i

,user_id,steam_id,item_id,item_name,playtime_forever
0,76561197970982479,76561197970982479,10,counter-strike,6.0
1,76561197970982479,76561197970982479,20,team fortress classic,0.0
2,76561197970982479,76561197970982479,30,day of defeat,7.0
3,76561197970982479,76561197970982479,40,deathmatch classic,0.0
4,76561197970982479,76561197970982479,50,half-life: opposing force,0.0
...,...,...,...,...,...
5094077,76561198329548331,76561198329548331,346330,brainbread 2,0.0
5094078,76561198329548331,76561198329548331,373330,all is dust,0.0
5094079,76561198329548331,76561198329548331,388490,one way to die: steam edition,3.0
5094080,76561198329548331,76561198329548331,521570,you have 10 seconds 2,4.0


In [159]:
df_i['items_count'] = df_i.groupby('user_id')['item_id'].transform('count')
df_i

,user_id,steam_id,item_id,item_name,playtime_forever,items_count
0,76561197970982479,76561197970982479,10,counter-strike,6.0,277
1,76561197970982479,76561197970982479,20,team fortress classic,0.0,277
2,76561197970982479,76561197970982479,30,day of defeat,7.0,277
3,76561197970982479,76561197970982479,40,deathmatch classic,0.0,277
4,76561197970982479,76561197970982479,50,half-life: opposing force,0.0,277
...,...,...,...,...,...,...
5094077,76561198329548331,76561198329548331,346330,brainbread 2,0.0,7
5094078,76561198329548331,76561198329548331,373330,all is dust,0.0,7
5094079,76561198329548331,76561198329548331,388490,one way to die: steam edition,3.0,7
5094080,76561198329548331,76561198329548331,521570,you have 10 seconds 2,4.0,7


In [160]:
cantidad_items = df_i[['items_count', 'user_id', 'item_id']]
cantidad_items

,items_count,user_id,item_id
0,277,76561197970982479,10
1,277,76561197970982479,20
2,277,76561197970982479,30
3,277,76561197970982479,40
4,277,76561197970982479,50
...,...,...,...
5094077,7,76561198329548331,346330
5094078,7,76561198329548331,373330
5094079,7,76561198329548331,388490
5094080,7,76561198329548331,521570


<br>
<br>

#### Se hace el merge de los primeros dos dataframes

In [161]:
df_user_data = cantidad_items.merge(dinero_gastado, on='item_id', how='left')
df_user_data

,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,20,4.99
2,277,76561197970982479,30,4.99
3,277,76561197970982479,40,4.99
4,277,76561197970982479,50,4.99
...,...,...,...,...
5094077,7,76561198329548331,346330,0.00
5094078,7,76561198329548331,373330,NaN
5094079,7,76561198329548331,388490,0.00
5094080,7,76561198329548331,521570,0.00


<br>
<br>

Se rellenan los valores nulos

In [162]:
df_user_data['price'] = df_user_data['price'].fillna(0.0)


Se hace la suma del dinero gastado por usuario, agrupando por user_id. La copia que se realiza es para poder hacer el merge con el 3er dataframe necesario.

In [163]:
df_user_data_parcial = df_user_data.groupby('user_id').agg({'price': 'sum', 'items_count': 'first'}).reset_index()

# Cambiar el nombre de la columna 'precio' a 'dinero_gastado'
df_user_data_parcial = df_user_data_parcial.rename(columns={'price': 'total_spent'})



In [164]:
df_user_data_parcial

,user_id,total_spent,items_count
0,--000--,397.78,58
1,--ace--,151.83,44
2,--ionex--,99.93,23
3,-2SV-vuLB-Kg,427.50,68
4,-404PageNotFound-,1509.32,149
...,...,...,...
70907,zzonci,19.98,5
70908,zzoptimuszz,64.98,61
70909,zzydrax,99.94,13
70910,zzyfo,813.53,84


<br>
<br>

Se obtienen las columnas 'item_id', 'recommend' y 'user_id' del Dataframe user_reviews

In [165]:
recomienda_items = df_r[['recommend', 'user_id', 'item_id']]


In [166]:
recomienda_items

,recommend,user_id,item_id
0,1,76561197970982479,1250
1,1,76561197970982479,22200
2,1,76561197970982479,43110
3,1,js41637,251610
4,1,js41637,227300
...,...,...,...
58425,1,76561198312638244,70
58426,1,76561198312638244,362890
58427,1,LydiaMorley,273110
58428,1,LydiaMorley,730


<br>
<br>

Se calcula el porcentaje de recomendacion por cada usuario

In [167]:
total_count = recomienda_items.groupby('user_id').size()

In [168]:
# Calcular el porcentaje de recommend True por user_id
total_count = recomienda_items.groupby('user_id').size()
recommend_true_count = recomienda_items[recomienda_items['recommend'] == 1].groupby('user_id').size() 

# Llenar con 0 los valores faltantes
recommend_true_count = recommend_true_count.reindex(total_count.index, fill_value=0)

# Calcular el porcentaje
percentage = (recommend_true_count / total_count) * 100

# Crear un nuevo dataframe con los resultados
recomendacion = pd.DataFrame({'user_id': percentage.index, 'recommend': percentage.values})
recomendacion


,user_id,recommend
0,--000--,100.0
1,--ace--,100.0
2,--ionex--,100.0
3,-2SV-vuLB-Kg,100.0
4,-Azsael-,100.0
...,...,...
25452,zwanzigdrei,100.0
25453,zy0705,100.0
25454,zynxgameth,100.0
25455,zyr0n1c,100.0


<br>
<br>

#### Se hace el segundo merge de los Dataframes creados

In [169]:
df_user_data_final = df_user_data_parcial.merge(recomendacion, on='user_id', how='left')
df_user_data_final['recommend'].fillna(0, inplace=True)
df_user_data_final

,user_id,total_spent,items_count,recommend
0,--000--,397.78,58,100.0
1,--ace--,151.83,44,100.0
2,--ionex--,99.93,23,100.0
3,-2SV-vuLB-Kg,427.50,68,100.0
4,-404PageNotFound-,1509.32,149,0.0
...,...,...,...,...
70907,zzonci,19.98,5,0.0
70908,zzoptimuszz,64.98,61,100.0
70909,zzydrax,99.94,13,0.0
70910,zzyfo,813.53,84,0.0


<br>
<br>

#### Se guarda como parquet

In [170]:
# Convierte el DataFrame a un objeto Table de PyArrow
table = pa.Table.from_pandas(df_user_data_final)

# Define las opciones de compresión con Snappy
compression = 'None'

# Guarda el objeto Table en formato Parquet con compresión Snappy
pq.write_table(table, "datasets\def_user_data.parquet", compression=compression)


<br>
<br>

## 03 Def user_for_genre

<br>

#### Usuario que acumula mas horas jugadas por genero dado y lista de acumulación de horas jugadas por año de lanzamiento.

<br>
<br>

Se obtienen las columnas 'playtime_forever', 'user_id', e 'item_id' del dataset de users_items, recordando que los valores de 'playtime_forever' ya se encuentran expresados en horas.

In [77]:
df_i

,user_id,item_id,item_name,playtime_forever,items_count
0,76561197970982479,10,counter-strike,6.0,277
1,76561197970982479,20,team fortress classic,0.0,277
2,76561197970982479,30,day of defeat,7.0,277
3,76561197970982479,40,deathmatch classic,0.0,277
4,76561197970982479,50,half-life: opposing force,0.0,277
...,...,...,...,...,...
5094077,76561198329548331,346330,brainbread 2,0.0,7
5094078,76561198329548331,373330,all is dust,0.0,7
5094079,76561198329548331,388490,one way to die: steam edition,3.0,7
5094080,76561198329548331,521570,you have 10 seconds 2,4.0,7


In [171]:
df_user = df_i[['playtime_forever', 'user_id', 'item_id']]
df_user

,playtime_forever,user_id,item_id
0,6.0,76561197970982479,10
1,0.0,76561197970982479,20
2,7.0,76561197970982479,30
3,0.0,76561197970982479,40
4,0.0,76561197970982479,50
...,...,...,...
5094077,0.0,76561198329548331,346330
5094078,0.0,76561198329548331,373330
5094079,3.0,76561198329548331,388490
5094080,4.0,76561198329548331,521570


<br>
<br>

Se obtienen las columnas 'item_id', 'release_year' y 'genre' del dataset steam_games

In [172]:
df_genero = df_g[['release_year', 'genres', 'item_id']]
df_genero

,release_year,genres,item_id
0,2018,action,761140
0,2018,casual,761140
0,2018,indie,761140
0,2018,simulation,761140
0,2018,strategy,761140
...,...,...,...
22525,2018,indie,610660
22525,2018,racing,610660
22525,2018,simulation,610660
22526,2017,casual,658870


<br>
<br>

#### Se realiza el merge de los datasets a traves del item_id

In [173]:
df_user_genre = df_genero.merge(df_user, on='item_id')
df_user_genre

,release_year,genres,item_id,playtime_forever,user_id
0,1997,action,282010,5.0,UTNerd24
1,1997,action,282010,0.0,I_DID_911_JUST_SAYING
2,1997,action,282010,0.0,76561197962104795
3,1997,action,282010,0.0,r3ap3r78
4,1997,action,282010,13.0,saint556
...,...,...,...,...,...
9495087,2004,action,80,0.0,76561198273508956
9495088,2004,action,80,0.0,76561198282090798
9495089,2004,action,80,0.0,943525
9495090,2004,action,80,9.0,76561198283312749


In [174]:
df_user_genre.value_counts("genres")

genres
action                     2688577
indie                      1668940
adventure                  1120627
rpg                         875449
strategy                    785121
free to play                702324
simulation                  518920
casual                      464286
massively multiplayer       340456
early access                120312
sports                       98616
racing                       94440
utilities                     4678
design and illustration       3767
animation and modeling        2526
web publishing                2064
education                     1437
software training             1394
video production               982
photo editing                  119
audio production                57
Name: count, dtype: int64

<br><br>

Reducir el tamaño del dataset en muestras por genero proporcional (para reducir tamaño y poder deployarlo)

In [ ]:
# Paso 1: Calcular la proporción de cada género
genero_counts = df_user_genre['genres'].value_counts()
total_registros = len(df_user_genre)
proporciones = (genero_counts / total_registros).to_dict()
proporciones



# Paso 2: Determinar cuántos registros se deben seleccionar para cada género
total_muestra = 100000  # Número total de registros deseados
muestra_genero = {genres: int(proporciones[genres] * total_muestra) for genres in proporciones}
muestra_genero

# Paso 3: Muestrear datos por género de forma proporcional
df_reducido = pd.DataFrame()

for genres, n_muestra in muestra_genero.items():
    subset_genero = df_user_genre[df_user_genre['genres'] == genres]
    muestra_genero = subset_genero.sample(n=n_muestra, random_state=42)  # Puedes ajustar el random_state
    df_reducido = pd.concat([df_reducido, muestra_genero])
df_reducido

# Paso 4: Seleccionar usuarios aleatoriamente
usuarios_unicos = df_reducido['user_id'].unique()
usuarios_muestra = pd.Series(usuarios_unicos).sample(frac=0.5, random_state=42).tolist()  # Ajusta el frac según lo que necesites

df_final = df_reducido[df_reducido['user_id'].isin(usuarios_muestra)]

df_user_genre = df_final

<br>
<br>

#### Se guarda como parquet

In [175]:
# Convierte el DataFrame a un objeto Table de PyArrow
table = pa.Table.from_pandas(df_user_genre)

# Define las opciones de compresión con Snappy
compression = 'None'

# Guarda el objeto Table en formato Parquet con compresión Snappy
pq.write_table(table, "datasets\\def_user_for_genre.parquet", compression=compression)

<br>
<br>

## 4) def best_developer_year

<br>

#### Devuelve top 3 de desarrolladora con más recomendaciones por usuario, por el año dado.

<br>
<br>

Se obtienen las columnas 'developer', 'release_year', e 'item_id' del dataset de steam_games.

In [176]:
df_g["developer"] = df_g["developer"].str.lower()
df_g

,item_id,app_name,release_year,price,developer,genres
0,761140,Lost Summoner Kitty,2018,4.99,kotoshiro,action
0,761140,Lost Summoner Kitty,2018,4.99,kotoshiro,casual
0,761140,Lost Summoner Kitty,2018,4.99,kotoshiro,indie
0,761140,Lost Summoner Kitty,2018,4.99,kotoshiro,simulation
0,761140,Lost Summoner Kitty,2018,4.99,kotoshiro,strategy
...,...,...,...,...,...,...
22525,610660,Russian Roads,2018,1.99,laush dmitriy sergeevich,indie
22525,610660,Russian Roads,2018,1.99,laush dmitriy sergeevich,racing
22525,610660,Russian Roads,2018,1.99,laush dmitriy sergeevich,simulation
22526,658870,EXIT 2 - Directions,2017,4.99,"xropi,stev3ns",casual


In [177]:
df_g = df_g.reset_index(drop=True)

In [178]:
df_dev = df_g[['release_year', 'developer', 'item_id']]
df_dev

,release_year,developer,item_id
0,2018,kotoshiro,761140
1,2018,kotoshiro,761140
2,2018,kotoshiro,761140
3,2018,kotoshiro,761140
4,2018,kotoshiro,761140
...,...,...,...
55601,2018,laush dmitriy sergeevich,610660
55602,2018,laush dmitriy sergeevich,610660
55603,2018,laush dmitriy sergeevich,610660
55604,2017,"xropi,stev3ns",658870


<br>
<br>

Se obtienen las columnas 'recommend', 'user_id' e 'item_id' del dataset de user_reviews.

In [179]:
df_recomen = df_r[['recommend', 'user_id', 'item_id']]
df_recomen

,recommend,user_id,item_id
0,1,76561197970982479,1250
1,1,76561197970982479,22200
2,1,76561197970982479,43110
3,1,js41637,251610
4,1,js41637,227300
...,...,...,...
58425,1,76561198312638244,70
58426,1,76561198312638244,362890
58427,1,LydiaMorley,273110
58428,1,LydiaMorley,730


<br>
<br>

#### Se realiza el merge de los datasets a traves del item_id.

In [180]:
df_best_developer = df_dev.merge(df_recomen, on='item_id')
df_best_developer

,release_year,developer,item_id,recommend,user_id
0,1997,stainless games ltd,282010,1,InstigatorAU
1,1997,stainless games ltd,282010,1,InstigatorAU
2,1997,stainless games ltd,282010,1,InstigatorAU
3,1998,valve,70,1,EizanAratoFujimaki
4,1998,valve,70,1,GamerFag
...,...,...,...,...,...
120668,2004,valve,80,0,76561198023508728
120669,2004,valve,80,1,Lone_walker
120670,2004,valve,80,1,green290
120671,2004,valve,80,1,174gamecuman700kngkakak


In [181]:
# Elimino los registros con release_year = sin_dato_fecha 
df_best_developer.drop(df_best_developer[df_best_developer['release_year'] == "Sin_dato_fecha"].index, inplace=True)

# convierto la columna a entero, para poder utilizar bien la función 
df_best_developer['release_year'] = df_best_developer['release_year'].astype(int)

<br>
<br>

#### Se carga como parquet

In [115]:
# Convierte el DataFrame a un objeto Table de PyArrow
table = pa.Table.from_pandas(df_best_developer)

# Define las opciones de compresión con Snappy
compression = 'None'

# Guarda el objeto Table en formato Parquet con compresión Snappy
pq.write_table(table, "datasets\\def_best_developer_year.parquet", compression=compression)

,release_year,developer,recommend
3,1998,valve,1
4,1998,valve,1
5,1998,valve,1
6,1998,valve,1
7,1998,valve,1
...,...,...,...
120668,2004,valve,0
120669,2004,valve,1
120670,2004,valve,1
120671,2004,valve,1


In [125]:
# df_best_developer2 = df_best_developer[["release_year","developer","recommend"]]
# df_best_developer2[df_best_developer2["developer"]=="valve"]

# df_best_developer2 = df_best_developer2.groupby(['release_year', 'developer'], as_index=False)['recommend'].sum()

# Renombrar la columna 'recommend' a 'sum_recommend'
# df_best_developer2 = df_best_developer2.rename(columns={'recommend': 'sum_recommend'})

# df_best_developer2


# Convierte el DataFrame a un objeto Table de PyArrow
# table = pa.Table.from_pandas(df_best_developer2)

# Define las opciones de compresión con Snappy
# compression = 'None'

# Guarda el objeto Table en formato Parquet con compresión Snappy
# pq.write_table(table, "datasets\\def_best_developer_year2.parquet", compression=compression)

,release_year,developer,sum_recommend
0,1989,"microprose software, inc",2
1,1990,id software,3
2,1990,lucasarts,1
3,1991,3d realms (apogee software),1
4,1992,3d realms (apogee software),2
...,...,...,...
2620,2017,targem games,4
2621,2017,"tlön studios,ilex games",6
2622,2017,trendy entertainment,185
2623,2017,"tt games,traveller's tales",10


<br>
<br>

## 5) def developer_reviews_analysis

<br>

#### Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

<br>
<br>

Se obtienen las columnas 'sentiment_analysis' e 'item_id' del dataset user_reviews

In [127]:
df_analisis = df_r[['sentiment_analysis', 'item_id']]
df_analisis

,sentiment_analysis,item_id
0,2,1250
1,1,22200
2,2,43110
3,2,251610
4,2,227300
...,...,...
58425,2,70
58426,2,362890
58427,2,273110
58428,2,730


<br>
<br>

Se obtienen las columnas 'developer' e 'item_id' del dataset steam_games

In [128]:
df_develop = df_g[['developer', 'item_id']]
df_develop

,developer,item_id
0,kotoshiro,761140
1,kotoshiro,761140
2,kotoshiro,761140
3,kotoshiro,761140
4,kotoshiro,761140
...,...,...
55601,laush dmitriy sergeevich,610660
55602,laush dmitriy sergeevich,610660
55603,laush dmitriy sergeevich,610660
55604,"xropi,stev3ns",658870


<br>
<br>

#### Se realiza el merge de los datasets a traves del item_id.

In [129]:
df_developer_reviews = df_develop.merge(df_analisis, on='item_id')
df_developer_reviews

,developer,item_id,sentiment_analysis
0,stainless games ltd,282010,1
1,stainless games ltd,282010,1
2,stainless games ltd,282010,1
3,valve,70,1
4,valve,70,0
...,...,...,...
120668,valve,80,1
120669,valve,80,2
120670,valve,80,1
120671,valve,80,1


<br>
<br>

#### Se carga como parquet

In [130]:
# Convierte el DataFrame a un objeto Table de PyArrow
table = pa.Table.from_pandas(df_developer_reviews)

# Define las opciones de compresión con Snappy
compression = 'None'

# Guarda el objeto Table en formato Parquet con compresión Snappy
pq.write_table(table, "datasets\\def_developer_reviews_analysis.parquet", compression=compression)

In [ ]:
# df_developer_reviews2 = df_developer_reviews[["developer","sentiment_analysis"]]

# Filtrar el DataFrame para eliminar los registros con sentiment_analysis igual a 1
# df_developer_reviews2 = df_developer_reviews2[df_developer_reviews2['sentiment_analysis'] != 1]

# Contar las ocurrencias de sentiment_analysis igual a 2 por developer
# sentiment_positive = df_developer_reviews2[df_developer_reviews2['sentiment_analysis'] == 2].groupby('developer').size().reset_index(name='sentiment_positive')

# Contar las ocurrencias de sentiment_analysis igual a 0 por developer
# sentiment_negative = df_developer_reviews2[df_developer_reviews2['sentiment_analysis'] == 0].groupby('developer').size().reset_index(name='sentiment_negative')

# Unir los conteos por developer
# df_developer_reviews2 = pd.merge(sentiment_positive, sentiment_negative, on='developer', how='outer').fillna(0)

# Convertir las columnas sentiment_positive y sentiment_negative a enteros
# df_developer_reviews2['sentiment_positive'] = df_developer_reviews2['sentiment_positive'].astype(int)
# df_developer_reviews2['sentiment_negative'] = df_developer_reviews2['sentiment_negative'].astype(int)
# df_developer_reviews2

# Convierte el DataFrame a un objeto Table de PyArrow
# table = pa.Table.from_pandas(df_developer_reviews)

# Define las opciones de compresión con Snappy
# compression = 'None'

# Guarda el objeto Table en formato Parquet con compresión Snappy
# pq.write_table(table, "datasets\\def_developer_reviews_analysis2.parquet", compression=compression)

,developer,sentiment_positive,sentiment_negative
0,"10th art studio,adventure productions",2,2
1,10tons ltd,3,0
2,11 bit studios,57,72
3,14° east,1,1
4,17-bit,4,0
...,...,...,...
1769,trerplay,0,3
1770,united independent entertainment gmbh,0,2
1771,wzogi,0,2
1772,yeaboing,0,2


<br>
<br>

# Modelo de ML para el sistema de recomendación 

<br>

#### Utiliza relación item-item (el **input** es un juego y el **output** es una lista de juegos recomendados), aplicando "similitud por coseno"

def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

<br>
<br>

Se analizan los Dataframes para seleccionar las variables

In [1]:
df_r.head(30)

NameError: name 'df_r' is not defined

In [183]:
df_g.head()

,item_id,app_name,release_year,price,developer,genres
0,761140,Lost Summoner Kitty,2018,4.99,kotoshiro,action
1,761140,Lost Summoner Kitty,2018,4.99,kotoshiro,casual
2,761140,Lost Summoner Kitty,2018,4.99,kotoshiro,indie
3,761140,Lost Summoner Kitty,2018,4.99,kotoshiro,simulation
4,761140,Lost Summoner Kitty,2018,4.99,kotoshiro,strategy


<br>
<br>

Se filtran las columnas necesarias

In [184]:
df_pre_modelo_g = df_g[['genres', 'app_name', 'item_id', 'developer']]
df_pre_modelo_g

,genres,app_name,item_id,developer
0,action,Lost Summoner Kitty,761140,kotoshiro
1,casual,Lost Summoner Kitty,761140,kotoshiro
2,indie,Lost Summoner Kitty,761140,kotoshiro
3,simulation,Lost Summoner Kitty,761140,kotoshiro
4,strategy,Lost Summoner Kitty,761140,kotoshiro
...,...,...,...,...
55601,indie,Russian Roads,610660,laush dmitriy sergeevich
55602,racing,Russian Roads,610660,laush dmitriy sergeevich
55603,simulation,Russian Roads,610660,laush dmitriy sergeevich
55604,casual,EXIT 2 - Directions,658870,"xropi,stev3ns"


In [185]:
df_pre_modelo_r = df_r[['item_id', 'recommend']]
df_pre_modelo_r

,item_id,recommend
0,1250,1
1,22200,1
2,43110,1
3,251610,1
4,227300,1
...,...,...
58425,70,1
58426,362890,1
58427,273110,1
58428,730,1


<br>
<br>

#### Se realiza el merge

In [186]:
df_modelo = df_pre_modelo_g.merge(df_pre_modelo_r, on='item_id')
df_modelo

,genres,app_name,item_id,developer,recommend
0,action,Carmageddon Max Pack,282010,stainless games ltd,1
1,indie,Carmageddon Max Pack,282010,stainless games ltd,1
2,racing,Carmageddon Max Pack,282010,stainless games ltd,1
3,action,Half-Life,70,valve,1
4,action,Half-Life,70,valve,1
...,...,...,...,...,...
120668,action,Counter-Strike: Condition Zero,80,valve,0
120669,action,Counter-Strike: Condition Zero,80,valve,1
120670,action,Counter-Strike: Condition Zero,80,valve,1
120671,action,Counter-Strike: Condition Zero,80,valve,1


<br>
<br>

Se trabaja el Dataframe

In [189]:
# Calcular el porcentaje de recomendación agrupando por app_name
recommendation_percentage = (df_modelo.groupby('app_name')['recommend'].mean() * 100).round(1)

# Fusionar los resultados con el DataFrame original
df_filtrado = df_modelo.merge(recommendation_percentage, on='app_name')

# Eliminar los valores duplicados y quedarse con una fila por genres-app_name
df_modelo_recomendacion = df_filtrado.drop_duplicates(subset=['genres', 'app_name'])
df_modelo_recomendacion


,genres,app_name,item_id,developer,recommend_x,recommend_y
0,action,Carmageddon Max Pack,282010,stainless games ltd,1,100.0
1,indie,Carmageddon Max Pack,282010,stainless games ltd,1,100.0
2,racing,Carmageddon Max Pack,282010,stainless games ltd,1,100.0
3,action,Half-Life,70,valve,1,98.4
64,action,The Ship: Murder Party,2400,outerlight ltd.,1,95.7
...,...,...,...,...,...,...
120645,action,Deathmatch Classic,40,valve,1,100.0
120646,action,Half-Life: Blue Shift,130,gearbox software,1,100.0
120651,action,Day of Defeat,30,valve,1,75.0
120655,action,Unreal Tournament 2004: Editor's Choice Edition,13230,"epic games, inc.",1,100.0


<br>
<br>

Se elimina la columna recommend_x

In [190]:
# Eliminar la columna 'recommend_x'
df_modelo_recomendacion = df_modelo_recomendacion.drop('recommend_x', axis=1)

<br>
<br>

#### Se carga el dataframe

In [191]:
# Convierte el DataFrame a un objeto Table de PyArrow
table = pa.Table.from_pandas(df_modelo_recomendacion)

# Define las opciones de compresión con Snappy
compression = 'None'

# Guarda el objeto Table en formato Parquet con compresión Snappy
pq.write_table(table, "datasets\def_recomendacion_juego.parquet", compression=compression)